In [1]:
"""
Bonito Model template
"""

from pickletools import optimize
import numpy as np

from torch import  nn, optim
from bonito.nn import Permute, layers
import torch
from torch.nn.functional import log_softmax, ctc_loss
from torch.nn import Module, ModuleList, Sequential, Conv1d, BatchNorm1d, Dropout
import toml
import torch
import numpy as np
import os

from bonito.util import __models__, default_config, default_data, load_model
from bonito.util import  load_symbol, init, half_supported, permute
from bonito.training import load_state, Trainer
from fast_ctc_decode import beam_search, viterbi_search
from torch.utils.data import DataLoader, TensorDataset
from bonito.reader import Reader

from train_flipflop import gen_batch
from _bin_argparse import get_train_flipflop_parser
class Decoder(Module):
    """
    Decoder
    """
    def __init__(self, features, classes):
        super(Decoder, self).__init__()
        self.layers = Sequential(
            Conv1d(features, classes, kernel_size=1, bias=True),
            Permute([2, 0, 1])
        )

    def forward(self, x):
        return log_softmax(self.layers(x), dim=-1)

def decode_ref(encoded, labels):
    """
    Convert a integer encoded reference into a string and remove blanks
    """
    return ''.join(labels[e] for e in encoded.tolist() if e)

dirname = "/home/princezwang/software/bonito/bonito/models/dna_r9.4.1@v2"
if not os.path.isdir(dirname) and os.path.isdir(os.path.join(__models__, dirname)):
    dirname = os.path.join(__models__, dirname)
pretrain_file = os.path.join(dirname, 'config.toml')
config = toml.load(pretrain_file)
if 'lr_scheduler' in config:
    print(f"[ignoring 'lr_scheduler' in --pretrained config]")
    del config['lr_scheduler']
model = load_model(dirname, 'cpu')



# model = load_symbol(config, 'Model')(config)
# scores = model(torch.randn(260,1,1000))
# seqs = [model.decode(x) for x in permute(scores, 'TNC', 'NTC')]
decoder_ww = model.state_dict()["decoder.layers.0.weight"]
decoder_bias = model.state_dict()["decoder.layers.0.bias"]
decoder_ww = torch.cat([decoder_ww,decoder_ww[2].reshape(-1,48,1)])
decoder_bias = torch.cat([decoder_bias,decoder_bias[2].reshape(-1)],0)
decoder_m = Decoder(48,6)
param_new = {}
for key in decoder_m.state_dict().keys():
    param_new[key] = decoder_m.state_dict()[key]
    print(key, decoder_m.state_dict()[key].shape)
param_new['layers.0.weight'] = decoder_ww
param_new['layers.0.bias'] = decoder_bias
decoder_m.load_state_dict(param_new)
model.decoder = decoder_m
model.alphabet = ['N', 'A', 'C', 'G', 'T', 'M']

# for p in model.encoder.parameters():
#     p.requires_grad=False

optimizer = optim.Adam(filter(lambda p:p.requires_grad, model.parameters()), lr=1e-4)
main_batch = gen_batch(get_train_flipflop_parser().parse_args())
batch_list = list(main_batch)
batch_tensor, seqref, seqlen = batch_list[0][0],batch_list[0][1],batch_list[0][2]
losses = []

for i in range(100):
    optimizer.zero_grad()
    tmp = permute(batch_tensor,[0,1,2],[1,2,0]) 
    scores = model(tmp)
    # seqs = [model.decode(x) for x in permute(scores, 'TNC', 'NTC')]
    loss = model.ctc_label_smoothing_loss(scores, seqref, seqlen)
    loss['total_loss'].backward()
    nn.utils.clip_grad_norm_(model.decoder.parameters(), max_norm=2, norm_type=2)
    optimizer.step()
    losses.append(loss['total_loss'])
    




/home/princezwang/.conda/envs/bonito_env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


layers.0.weight torch.Size([6, 48, 1])
layers.0.bias torch.Size([6])


usage: ipykernel_launcher.py [-h] [--size neurons] [--stride samples]
                             [--winlen WINLEN] [--adam beta1 beta2]
                             [--eps adjustment] [--niteration batches]
                             [--weight_decay penalty]
                             [--gradient_clip_num_mads num_MADs]
                             [--lr_max rate] [--lr_min rate] [--seed integer]
                             [--sharpen min max niter]
                             [--warmup_batches WARMUP_BATCHES]
                             [--lr_warmup rate] [--min_momentum MIN_MOMENTUM]
                             [--filter_max_dwell multiple]
                             [--filter_mean_dwell radius]
                             [--filter_min_pass_fraction fraction]
                             [--filter_path_buffer ratio] [--limit LIMIT]
                             [--reverse] [--sample_nreads_before_filtering n]
                             [--chunk_len_min samples]
       

SystemExit: 2

/home/princezwang/.conda/envs/bonito_env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3406: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
model